### Import Libraries 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
tf.config.set_visible_devices([], 'GPU')

from network import build_recurrent_model
from load_data import load_experiment

### Load Experiment Data
    - Arguments: 
        - dataset: {'mnist', 'fashion_mnist', 'imdb', 'cifar10', 'reuters'}
        - permute: {True/False}
        - orientation: {None, 'post', 'uniform'}

    - E.g. 1: load sequential mnist task
        -> (train_x, train_y), (test_x, test_y) = load_experiment('mnist', permute=False, orientation=None)

    - E.g. 2: load post noise orientation imdb task
        -> (train_x, train_y), (test_x, test_y) = load_experiment('imdb', permute=False, orientation='post')
        

In [ ]:
# E.g. 1: load sequential mnist task

dataset = 'mnist'
permute = False
orientation = None

(train_x, train_y), (test_x, test_y) = load_experiment(dataset, permute=permute, orientation=orientation)

print('******** TASK Data Details ********')
print(f'dataset: {dataset}')
print(f'permute: {permute}')
print(f'orientation: {orientation}')

print(f'train features dim: {train_x.shape}')
print(f'train labels dim: {train_y.shape}')
print(f'test features dim: {test_x.shape}')
print(f'test labels dim: {test_y.shape}')

### Generate Recurrent Model
    - Function: build_recurrent_model(name, time_horizon, ft_dim, hid_dim, output_dim, learning_rate, embed=False)
        
    - Arguments:
            - name (string):          { 'rnn', 'antisymmetric', 'exponential', 'lstm', 'gru', 'lipschitz', 'unicornn' }
            - time_horizon (int):     { time horizon of underlying task (i.e., input sequence length) }
            - ft_dim (int):           { dimension of input sequence element }
            - hid_dim (int):          { recurrrent dimension }
            - output_dim (int):       { dimension of output (i.e., number of class labels) }
            - learning_rate (float):  { optimizer learning rate }
            - embed (bool):           { boolean indicating whether the task requires an embedding layer (NLP tasks) }

    - Note: For NLP tasks involving the data sets 'imdb' and 'reuters', a word-embedding is learned. To accomodate this, we fix an embedding dimension to 50 (i.e., set `ft_dim=50` in model initialization). 

In [ ]:
K.clear_session()

arch = 'rnn'
embed = False if dataset in ['mnist', 'fashion_mnist', 'cifar10'] else True
T = int(train_x.shape[1])
ft_dim = 50 if embed else int(train_x.shape[2]) 
hid_dim = 128
out_dim = 1 if len(np.unique(train_y)) == 2 else len(np.unique(train_y))
learning_rate = 1e-4


model = build_recurrent_model(arch, T, ft_dim, hid_dim, out_dim, learning_rate, embed=embed)

print('embed:', embed)
rec_layer = model.layers[2].name if embed else model.layers[1].name
print(f'recurrent layer: {rec_layer}')
model.summary()

### Train Model

In [ ]:
model.fit(train_x, train_y, epochs=3, validation_data=(test_x,test_y))